In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
budget_file_path = r'../datasets/march_budget_expenditures/' # use your path
all_budget_files_path = glob.glob(budget_file_path + "/*.csv")
all_budget_files_path.sort()
date_rng = pd.date_range(start='03/01/2019', end='03/31/2019', freq='D')
budget_file_array =  np.asarray(all_budget_files_path)
filename_array = []
for i in range(len(all_budget_files_path)):
    filename_array.append(all_budget_files_path[i].split('/')[3])

In [4]:
budget_expenditure_files = []
for i in range(len(all_budget_files_path)):
    budget_expenditure_df = pd.read_csv(all_budget_files_path[i])
    budget_expenditure_df.replace(" ", np.NaN, inplace = True)
    budget_expenditure_df.fillna(method='ffill', inplace=True)
    budget_expenditure_df = budget_expenditure_df[~budget_expenditure_df.Dmdcd.str.contains("Total")]
    budget_expenditure_df = budget_expenditure_df[~budget_expenditure_df.Majnm.str.contains("Total")]
    budget_expenditure_df = budget_expenditure_df[~budget_expenditure_df.SoeCd.str.contains("Total")]
    budget_expenditure_df[['Major_Head_Code', 'Major_Head_Description']] = budget_expenditure_df['Majnm'].str.split(r"-", expand=True, n=1)
    budget_expenditure_df[['SOE_Code', 'SOE_Description']] = budget_expenditure_df['SoeCd'].str.split(r"-", expand=True, n=1)
    budget_expenditure_df.reset_index(inplace=True, drop = True)
    budget_expenditure_df.drop(['Majnm'], axis = 1, inplace = True)
    budget_expenditure_df.drop(['SoeCd'], axis = 1, inplace = True)
    budget_expenditure_df['Date'] = date_rng[i]
    budget_expenditure_df['Month'] = 'March'
    budget_expenditure_df =budget_expenditure_df[['Dmdcd',
                                                  'Major_Head_Code',
                                                  'Major_Head_Description',
                                                  'SOE_Code',
                                                  'SOE_Description',
                                                  'Date',
                                                  'Month',
                                                  'SANCTION',
                                                  'ADDITION',
                                                  'SAVING',
                                                  'REVISED']]
    budget_expenditure_files.append(budget_expenditure_df)
budget_expenditure_total_df = pd.concat(budget_expenditure_files, axis=0, ignore_index=True)

In [5]:
budget_expenditure_total_df.head()

,Dmdcd,Major_Head_Code,Major_Head_Description,SOE_Code,SOE_Description,Date,Month,SANCTION,ADDITION,SAVING,REVISED
0,01,2011,PARLIAMENT/STATE/U.T.LEGISLATU,01,SALARIES,2019-03-01,March,3045.73,13.43,438.78,2620.38
1,01,2011,PARLIAMENT/STATE/U.T.LEGISLATU,02,WAGES,2019-03-01,March,0.21,0.00,0.13,0.08
2,01,2011,PARLIAMENT/STATE/U.T.LEGISLATU,03,TRAVEL EXPENSES,2019-03-01,March,218.31,37.50,2.48,253.33
3,01,2011,PARLIAMENT/STATE/U.T.LEGISLATU,04,LIVERIES,2019-03-01,March,2.50,7.94,6.86,3.58
4,01,2011,PARLIAMENT/STATE/U.T.LEGISLATU,05,OFFICE EXPENSES,2019-03-01,March,92.43,57.00,0.16,149.27


In [6]:
budget_expenditure_total_df.to_csv('../datasets/prep_data/budget_expenditue_march.csv', index = False)